# 01 - Data Cleaning
Load raw data, clean, and save processed data.

In [ ]:
import dagshub
dagshub.init(repo_owner='yahiaehab10', repo_name='MLflow_demo_MF', mlflow=True)

In [ ]:
from sklearn.datasets import load_iris
import pandas as pd
from src.data_quality import clean_data, save_data

# Load and save raw data
iris = load_iris(as_frame=True)
df_raw = iris.frame.copy()
df_raw['target'] = iris.target
raw_path = '../data/raw/data.csv'
df_raw.to_csv(raw_path, index=False)

# Clean and save processed data
df_clean = clean_data(df_raw)
processed_path = '../data/processed/data.csv'
save_data(df_clean, processed_path)
print('Data cleaned and saved to', processed_path)